<a href="https://colab.research.google.com/github/lizaoh/smp_program_data/blob/main/smp2011_extract_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Top of Script

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pymupdf
!pip install pymupdf-layout
!pip install pymupdf4llm
# !pip install rapidfuzz
import glob
import os
import pathlib
import pymupdf
import pymupdf.layout
import pymupdf4llm
import re
import pandas as pd
import unicodedata
# from rapidfuzz import process, fuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 122.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 147.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 127.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 143.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.6 MB/s eta 0:00:00
  Attempting uninstall: PyMuPDF
    Found existing installation: PyMuPDF 1.26.7
    Uninstalling PyMuPDF-1.26.7:
      Successfully uninstalled PyMuPDF-1.26.7
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 kB 2.5 MB/s eta 0:00:00


In [3]:
pdfs_path = '/content/drive/MyDrive/math_psych_work/Conference Programs/'

# Functions
Created with help from GPT 5.2, but some are my own code just turned into a function.

In [206]:
AFFILIATION_KEYWORDS = [
    "University", "College", "Department", "Center", "Institute",
    "Laboratory", "School", "Hospital", "UC", "Centre", "Research",
    "Corporation", "Defence", "Université", "Universite", "Universiy"
]
AFFILIATION_KEYWORDS = re.compile(r'\b(' + '|'.join(AFFILIATION_KEYWORDS) + r')\b',
                                  re.I)

def looks_like_affiliation(chunk):
    return bool(AFFILIATION_KEYWORDS.search(chunk))

In [212]:
def normalize_whitespace(s: str) -> str:
    return " ".join(s.replace("\n", "").split())

In [213]:
def normalize_affiliations(entry: str) -> str:
    return re.sub(
        r'(University of California)\s*,?\s*'
        r'(Irvine|Davis|Berkeley|Los Angeles|San Diego|Santa Barbara|Santa Cruz|Riverside|Merced)',
        r'\1, \2',
        entry
    )

In [226]:
def split_authors_affiliations(entry: str) -> tuple[str, str]:
    entry = normalize_whitespace(entry)
    entry = normalize_affiliations(entry)

    # split only on AND / & / commas WITH SPACES
    tokens = re.split(r'\s+(?:and|&)\s+|,\s+(?=[A-Z])', entry)

    authors = []
    affiliations = []

    for token in tokens:
        token = token.strip()

        if looks_like_affiliation(token):
            affiliations.append(token)
        else:
            authors.append(token)

    return (
        ", ".join(authors),
        "; ".join(affiliations)
    )

In [221]:
def clean_text(text):
    if not text:
        return text

    text = re.sub(r' \n\n\d{1,3} \n\n', ' ', text)
    text = re.sub(r'\s*\n\s*', ' ', text)    # Replace newlines with spaces

    text = re.sub(r'-\s+', '', text)         # Get rid of "- "; will fix actual
                                             # hyphenated words manually

    text = re.sub(r'\s{2}', ' ', text)       # Collapse two adjacent spaces into one

    text = re.sub(r'\.\s*##.*$', '.', text,\
                  flags=re.DOTALL)           # Gets rid of extraneous text after
                                             # last sentence
    text = text.strip()
    text = fix_ligatures(text)

    return text

In [9]:
LIGATURE_MAP = {
    "ﬁ": "fi",
    "ﬂ": "fl",
    "ﬃ": "ffi",
    "ﬄ": "ffl",
    "ﬀ": "ff",
    "ﬅ": "ft",
    "ﬆ": "st",
    "Æ": 'ffi'
}

def fix_ligatures(text):
    # Replace known ligatures
    for bad, good in LIGATURE_MAP.items():
        text = text.replace(bad, good)

    # Replace any private-use ligature (common in PDFs)
    cleaned_chars = []
    for ch in text:
        name = unicodedata.name(ch, "")
        if "LIGATURE" in name.upper():
            # Try to break it apart: remove spaces and lowercase
            base = name.split("LIGATURE")[-1]
            base = base.replace(" ", "").lower()
            cleaned_chars.append(base)
        else:
            cleaned_chars.append(ch)

    return "".join(cleaned_chars)

# Program

Extracting to markdown doesn't show when text is bold or italic even though it is in the pdf, so just extracting as text.

145 entries total (3 plenary, 22 symposium talks, 101 normal talks, 19 posters).

## Grab text from the pdf

In [10]:
year = '2011'
program = pymupdf.open(pdfs_path + f'smp{year}_program.pdf')

In [11]:
# Extracting as markdown to more easily get title, authors, and affiliations
# Title is bold, affiliations are italic, etc.
program_text = pymupdf4llm.to_text(program)

In [174]:
program_text[14500:18000]

'laces on the Boston Avenue side of the campus. Also there are several good restaurants in the Davis Square area, which is a ten-minute walk away from campus on College Avenue. \n\n6 \n\nAbstracts For Plenary Talks \n\n(Plenary abstracts organized by day) \n\n(Plenary) \n\nSaturday, 1:45 Plenary Changing the State of the Observer in Visual Processing: Perceptual Learning and Attention. Barbara Anne Dosher, University of California, Irvine. Processing of visual inputs is fundamental to how we interact with the world. The quality of performance depends in complex ways on properties of the stimulus as well as limitations in the observers system. It may also depend on the changing state of the observer, including changing states of perceptual learning, attention, adaptation, and others. This talk focuses on the phenomenon of perceptual learning, which improves perceptual task performance in several distinctive ways. External noise tests and noisy ideal observer models characterize the effe

In [160]:
BEFORE_TITLE_PHRASES = ['Plenary', 'Cohen', 'Distler',
                        r'East Sophia Gordon(?: Hall)? Bldg\.',
                        'Alumni', r'Board [A-E] #[1-4]']
before_title_re = re.compile(
    r'(?:' + '|'.join(BEFORE_TITLE_PHRASES) + r')\s+'
)

poster_split = re.split(r'(?:Saturday|Sunday|Monday), \d{1,2}:\d{2}-\d{1,2}:\d{2}\s*',\
                         program_text)
talks_split = re.split(r'(?:Saturday|Sunday|Monday), \d{1,2}:\d{2}\s*',\
                           poster_split[0])
all_entries = talks_split[1:] + poster_split[1:]

split_entries = [
    part.strip()
    for entry in all_entries
    for part in before_title_re.split(entry)
    if part.strip()
][:-8]
split_entries[-1] = re.split(r' \n\nAuthor Index', split_entries[-1], 1)[0]

In [189]:
fixed_entries = []

for entry in split_entries:
  if '<==\n\n' in entry:
    fixed_entries.extend(entry.split('<==\n\n'))
  else:
    fixed_entries.append(entry)

In [190]:
filtered_entries = [entry for entry in fixed_entries if len(entry.strip()) > 750]

In [233]:
filtered_entries[:2]

['Changing the State of the Observer in Visual Processing: Perceptual Learning and Attention. Barbara Anne Dosher, University of California, Irvine. Processing of visual inputs is fundamental to how we interact with the world. The quality of performance depends in complex ways on properties of the stimulus as well as limitations in the observers system. It may also depend on the changing state of the observer, including changing states of perceptual learning, attention, adaptation, and others. This talk focuses on the phenomenon of perceptual learning, which improves perceptual task performance in several distinctive ways. External noise tests and noisy ideal observer models characterize the effects of perceptual learning (or attention) on an observer as a single system. Across a range of tasks, two independent mechanisms can be seen: tuning of the task relevant perceptual template to improve filtering (external noise exclusion) and enhancing the stimulus (reducing absolute threshold).

In [227]:
parsed_entries = []

for e, entry in enumerate(filtered_entries):
  no_trailing_junk = entry.split(".")[:-1]
  cleaned_entry = clean_text(".".join(no_trailing_junk))

  title, rest_of_entry = cleaned_entry.split('.', 1)
  auth_and_aff, abstract = re.split(r'(?<!\b[A-Z])\.\s+(?=[A-Z])', rest_of_entry, 1)

  authors, affiliations = split_authors_affiliations(auth_and_aff)
  cleaned_abstract = clean_text(abstract)

  parsed_entries.append({
    'year': year,
    'author(s)': authors,
    'affiliation(s)': affiliations,
    'title': title,
    'type': '',
    'abstract': cleaned_abstract + '.'  # add back last period
  })

In [234]:
parsed_entries[:2]

[{'year': '2011',
  'author(s)': 'Barbara Anne Dosher, Irvine',
  'affiliation(s)': 'University of California',
  'title': 'Changing the State of the Observer in Visual Processing: Perceptual Learning and Attention',
  'type': '',
  'abstract': 'Processing of visual inputs is fundamental to how we interact with the world. The quality of performance depends in complex ways on properties of the stimulus as well as limitations in the observers system. It may also depend on the changing state of the observer, including changing states of perceptual learning, attention, adaptation, and others. This talk focuses on the phenomenon of perceptual learning, which improves perceptual task performance in several distinctive ways. External noise tests and noisy ideal observer models characterize the effects of perceptual learning (or attention) on an observer as a single system. Across a range of tasks, two independent mechanisms can be seen: tuning of the task relevant perceptual template to impro

# Create df and convert to csv

In [229]:
df = pd.DataFrame(parsed_entries, columns=["year", "author(s)", "affiliation(s)", "title", "type", "abstract"])

In [230]:
df.head()

,year,author(s),affiliation(s),title,type,abstract
0,2011,"Barbara Anne Dosher, Irvine",University of California,Changing the State of the Observer in Visual P...,,Processing of visual inputs is fundamental to ...
1,2011,Yann LeCun,Courant Institute; New York University,Training Hierarchical Systems for Visual Perce...,,"Over the last few years, a surprising converge..."
2,2011,Janne V. Kujala,University of Jyvaskyla,Dependence of random variables on external fac...,,In this talk I will address two distinct topic...
3,2011,"F. Gregory Ashby, Santa Barbara",University of California,A Brief History of General Recognition Theory,,General recognition theory (GRT) was initially...
4,2011,Noah Silbert,University of Maryland,Two hierarchical Bayesian General Recognition ...,,General Recognition Theory (GRT) provides a po...


In [231]:
df.to_csv(f"/content/drive/MyDrive/math_psych_work/csv/smp{year}_program.csv", index=False)